In [ ]:
# processes data on projected heat increase due to urban land expansion

In [1]:
import os
import sys
import math
import warnings
import pandas as pd
import geopandas as gpd
from osgeo import gdal
from pathlib import Path
import glob
import numpy as np
import rasterio.mask
import rasterio
from rasterio.plot import show
from rasterio.warp import calculate_default_transform, reproject, Resampling

In [2]:
# Raw data folder. Change file path as needed
input_folder = Path(r'F:\World Bank\City Scan\data\Heat increase due to urban land expansion')

In [3]:
def reproj_heat(input_raster):
    filename = input_raster + '.tif'
    outfile = input_raster + '_4326.tif'
    with rasterio.open(input_folder / filename) as src:
        dst_crs = 'EPSG:4326'
        
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        with rasterio.open(input_folder / outfile, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)

In [4]:
shp_folder = Path('data/AOI')
cities = pd.read_csv('centroids.csv').city
output_folder = Path('data')

In [5]:
def clip_heat(input_raster):
    input_raster_name = input_raster + '_4326.tif'
    with rasterio.open(input_folder / input_raster_name) as src:
        out_image, out_transform = rasterio.mask.mask(
            src, features, crop=True)
        out_meta = src.meta.copy()
        
    out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform})
    
    out_file = city.replace(' ', '_').lower() + '_' + input_raster + '.tif'
    with rasterio.open(city / output_folder / out_file, "w", **out_meta) as dest:
        dest.write(out_image)

In [6]:
raster_list = ['urban-ssp1_day_sum', 'urban-ssp1_nig_sum',
               'urban-ssp2_day_sum', 'urban-ssp2_nig_sum',
               'urban-ssp3_day_sum', 'urban-ssp3_nig_sum']

In [7]:
for city in cities:
    aoi_name = city.replace(' ', '_').lower() + '_AOI.shp'
    shp = gpd.read_file(city / shp_folder / aoi_name)#.buffer(0.01)
    features = shp.geometry
    for raster in raster_list:
        clip_heat(raster)